In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

  !ls

Saving BBB_labels.txt to BBB_labels.txt
Saving BBB_features.csv to BBB_features.csv
User uploaded file "BBB_labels.txt" with length 6110 bytes
BBB_features.csv  BBB_labels.txt  sample_data
User uploaded file "BBB_features.csv" with length 3357142 bytes
BBB_features.csv  BBB_labels.txt  sample_data


In [2]:
import re
import numpy as np
import pandas as pd

df = pd.read_csv('BBB_features.csv', sep=';')
label = pd.read_csv('BBB_labels.txt', header=None, sep=' ')
newdf = pd.DataFrame(columns=df.columns)
logbb = []
permeate = []
for molname in df['Objects']:
    basename = re.sub(r'_c0', '', molname)
    if basename in label[0].values:
        lobBB = label[label[0] == basename][1].values[0]
        logbb.append(lobBB)
        if lobBB > 0:
            permeate.append(1)
        else:
            permeate.append(0)
        newdf = pd.concat([newdf, df[df['Objects'] == molname]])
    else:
        print('removing', molname)

df = newdf

<ipython-input-2-f75cd362ee4d>:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  newdf = pd.concat([newdf, df[df['Objects'] == molname]])


removing 1,1,1,2-tetrachloroethane_c1
removing 1,1,1,2-tetrachloroethane_c2
removing 1,1,1,2-tetrachloroethane_c3
removing 1,1,1,2-tetrachloroethane_c4
removing 1,1,1,2-tetrachloroethane_c5
removing 1,1,1,2-tetrachloroethane_c6
removing 1,1,1,2-tetrachloroethane_c7
removing 1,1,1,2-tetrachloroethane_c8
removing 1,4-divinyloxybutane_c10
removing 1,4-divinyloxybutane_c11
removing 1,4-divinyloxybutane_c12
removing 1,4-divinyloxybutane_c13
removing 1,4-divinyloxybutane_c14
removing 1,4-divinyloxybutane_c15
removing 1,4-divinyloxybutane_c1
removing 1,4-divinyloxybutane_c2
removing 1,4-divinyloxybutane_c3
removing 1,4-divinyloxybutane_c4
removing 1,4-divinyloxybutane_c5
removing 1,4-divinyloxybutane_c6
removing 1,4-divinyloxybutane_c7
removing 1,4-divinyloxybutane_c8
removing 1,4-divinyloxybutane_c9
removing 1-butanol_c10
removing 1-butanol_c11
removing 1-butanol_c12
removing 1-butanol_c13
removing 1-butanol_c14
removing 1-butanol_c15
removing 1-butanol_c16
removing 1-butanol_c17
removing 1-

In [3]:
print('logBB', len(logbb))
print('permeate', len(permeate))
print(df.shape)

logBB 330
permeate 330
(330, 128)


In [4]:
X = df.drop(['Objects'], axis=1)
X = X.dropna(axis=1)
# remove constant columns
X = X.loc[:, (X != X.iloc[0]).any()]
# remove duplicate columns
X = X.T.drop_duplicates().T
y_regression = np.array(logbb)
y_classification = np.array(permeate)

print('X.shape', X.shape)
print('y_regression.shape', y_regression.shape)
print('y_classification.shape', y_classification.shape)

X.shape (330, 120)
y_regression.shape (330,)
y_classification.shape (330,)


In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_regression_train, \
    y_regression_test, y_classification_train, \
    y_classification_test = \
    train_test_split(X, y_regression, y_classification, \
                     test_size=0.2, random_state=42)
print('X_train.shape', X_train.shape)
print('y_regression_train.shape', y_regression_train.shape)
print('y_classification_train.shape', y_classification_train.shape)
print('X_test.shape', X_test.shape)
print('y_regression_test.shape', y_regression_test.shape)
print('y_classification_test.shape', y_classification_test.shape)

ntrees = 50

X_train.shape (264, 120)
y_regression_train.shape (264,)
y_classification_train.shape (264,)
X_test.shape (66, 120)
y_regression_test.shape (66,)
y_classification_test.shape (66,)


In [6]:
import timeit

start = timeit.default_timer()
rf_regression = RandomForestRegressor(n_estimators=ntrees, random_state=42)
rf_regression.fit(X_train, y_regression_train)
y_regression_pred = rf_regression.predict(X_train)
msetrain = mean_squared_error(y_regression_train, y_regression_pred)
r2train = rf_regression.score(X_train, y_regression_train)
print('  Train R2 %5.3f'%(r2train))
print(' Train MSE %8.6e'%(msetrain))
endtrain = timeit.default_timer()
print('Time train %8.3f s'%(endtrain - start))

  Train R2 0.946
 Train MSE 2.406089e-02
Time train    0.949 s


In [7]:
# build a calssification RandomForrest model
start = timeit.default_timer()
rf_classification = RandomForestClassifier(n_estimators=ntrees, random_state=42)
rf_classification.fit(X_train, y_classification_train)
y_classification_pred = rf_classification.predict(X_train)
accuracytrain = accuracy_score(y_classification_train, y_classification_pred)
print('Train Accuracy %5.3f'%(accuracytrain))
endtrain = timeit.default_timer()
print('Time train %8.3f s'%(endtrain - start))

Train Accuracy 1.000
Time train    0.162 s
